In [1]:
import polars as pl
import pandas as pd
import datetime 
from datetime import datetime as dt

import xmltodict
import json

In [5]:
import pandas as pd

df=pd.read_csv(r'root_sample.csv', on_bad_lines='skip')

C:\Users\Pham Quy Nam\AppData\Local\Temp\ipykernel_22988\3018620180.py:3: DtypeWarning: Columns (1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,41,42,43,45,46,48,50,53,55,57,60,62,63,64,67,69,70,71,72,73,74,75,76,77,78,79,80,81,82,84,85,86,87,88,89,90,91,92,93,94,96,98,99,101,103,106,108,113,115,118,120,121,122,123,124,125,126,127,128,129,130,131,132,138,141,143,144,145,147,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,180,181,182,183,184,185,186,187,188,189,190,191,194,196,199,200,202,203,204,205,206,207,208,209,210,211,212,213,214,217,218,219,220,221,222,223,224,225,226) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r'root_sample.csv', on_bad_lines='skip')


In [6]:
df

,InquiredOperation.CBContractCode,InquiredOperation.TypeOfFinancing,GeneralData.CurrencyCode,GeneralData.TotalNumberOfContract,GeneralData.NumberOfReportingInstitution,GeneralData.WorstRecentStatus,Contract.Instalments.Summary.NumberOfRequested,Contract.Instalments.Summary.NumberOfLiving,Contract.Instalments.Summary.NumberOfRefused,Contract.Instalments.Summary.NumberOfRenounced,...,Contract.NonInstalments.NotGrantedContract.FIContractCode,Contract.Cards.GrantedContract.CardsGuarantees.CBSubjectCode,Contract.Cards.GrantedContract.CardsGuarantees.FIGuaranteeCode,Contract.Cards.GrantedContract.CardsGuarantees.GuaranteedAmount,Contract.Cards.GrantedContract.CardsGuarantees.GuarantorDescription,Contract.Cards.GrantedContract.CardsGuarantees.CodeGuaranteeType,Contract.Cards.GrantedContract.CardsGuarantees.RealGuarantee.AssetCode,Contract.Cards.GrantedContract.CardsGuarantees.RealGuarantee.AssetDescription,Contract.Cards.GrantedContract.CardsGuarantees.RealGuarantee.DateOfPledgeOrMortgage,Contract.Cards.GrantedContract.CardsGuarantees.RealGuarantee.ActualDateOfReleaseOfMortgage
0,927731262,31,VND,11,6,1,3,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,L27735996,31,VND,3,1,NaN,2,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M27733334,31,VND,18,8,1,9,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M27733337,31,VND,3,2,1,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E27733495,31,VND,1,1,NaN,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27003,127828709,31.0,VND,20,6,1.0,11,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27004,N27838836,31.0,VND,0,0,NaN,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27005,Q27840836,31.0,VND,0,0,NaN,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27006,M27838245,31.0,VND,0,0,NaN,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.info(verbose=True)

In [ ]:
import prefect

In [ ]:
from src.utils import *

In [ ]:
import os
os.getcwd()

In [ ]:
df = pd.read_csv(r'pcb_2024_T01_T06.csv', nrows=1000)

In [ ]:
def conso_id(id1, id2):
    if id1 == "'":
        final_id = id2
    elif id2 == "'":
        final_id = id1
    else:
        if id1 == id2:
            final_id = id1
        else:
            final_id = id1+"_"+id2
    return final_id

In [ ]:
def get_log(row):
    full_message = xmltodict.parse(row)
    try:
        to_dict = full_message['MGResponse']['RI_Req_Output']['CreditHistory']
    except:
        to_dict = full_message['MGResponse']['CI_Req_Output']['CreditHistory']
    return str(to_dict)

In [ ]:
df.columns = ['id1', 'id2', 'name', 'address', 'log', 'process_date']

In [ ]:
df['process_date'] = df['process_date'].astype(str)

In [ ]:
df['cccd'] = df[['id1', 'id2']].apply(lambda 
                                      row: conso_id(row.id1, row.id2),
                                      axis=1
                                     )

In [ ]:
df['pcb'] = df['log'].apply(lambda row: get_log(row))

In [ ]:
from pip._vendor.rich.progress import (
    Progress, SpinnerColumn, TimeElapsedColumn
)

def parse_data(process_name, df_in, field, id_lst):
    try:
        df_holder = []
        total_rows = len(df_in)

        with Progress(SpinnerColumn(), *Progress.get_default_columns(), TimeElapsedColumn(), transient=False) as progress:
            task = progress.add_task(f"Parsing {process_name}", total=total_rows)
            def process_row(row):    
                explode_nest(df_holder, id_lst, row, row[field[0]])
                progress.update(task, advance=1)
                
            df_in[id_lst + field].apply(process_row, axis=1)
            df_out = pd.concat(df_holder)
            df_out = df_out.replace('null', np.nan)
    except KeyError:
        df_out = pd.DataFrame()
    return df_out

def gen_id_level2(df):
    return df['cccd'].astype(str) + '_' + df['CommonData.CBContractCode']

def treat_dtype(df, col):
    try:
        df[col] = df[col].astype(str)
    except KeyError:
        pass

In [ ]:
field = ['pcb']
id_col_list = ['cccd', 'process_date']

df_root = parse_data('root', df, field, id_col_list)

In [ ]:
dtype_treat_lst = ['Contract.Instalments.GrantedContract',
                  'Contract.Cards.GrantedContract',
                  'Contract.NonInstalments.GrantedContract',
                  'Contract.Instalments.NotGrantedContract',
                  'Contract.Cards.NotGrantedContract',
                  'Contract.NonInstalments.NotGrantedContract']
for i in dtype_treat_lst:
    treat_dtype(df_root, i)

In [ ]:
id_col_list = ['cccd', 'process_date']

contract_level_noninstall = parse_data('nonins', df_root, ['Contract.NonInstalments.GrantedContract'], id_col_list)
contract_level_install = parse_data('ins', df_root, ['Contract.Instalments.GrantedContract'], id_col_list)
contract_level_card = parse_data('card', df_root, ['Contract.Cards.GrantedContract'], id_col_list)

not_grant_install = parse_data('ng_ins', df_root, ['Contract.Instalments.NotGrantedContract'], id_col_list)
not_grant_noninstall = parse_data('ng_nonins', df_root, ['Contract.NonInstalments.NotGrantedContract'], id_col_list)
not_grant_card = parse_data('ng_card', df_root, ['Contract.Cards.NotGrantedContract'], id_col_list)

In [ ]:
contract_level_noninstall['loan_code_lv2'] = gen_id_level2(contract_level_noninstall)
contract_level_install['loan_code_lv2'] = gen_id_level2(contract_level_install)
contract_level_card['loan_code_lv2'] = gen_id_level2(contract_level_card)

treat_dtype(contract_level_install, 'Profiles')
treat_dtype(contract_level_install, 'InstGuarantees')
treat_dtype(contract_level_card, 'Profiles')
treat_dtype(contract_level_card, 'CardsGuarantees')
treat_dtype(contract_level_noninstall, 'Profiles')

In [ ]:
contract_level_card = contract_level_card[~contract_level_card['loan_code_lv2'].isna()]
contract_level_noninstall = contract_level_noninstall[~contract_level_noninstall['loan_code_lv2'].isna()]
contract_level_install = contract_level_install[~contract_level_install['loan_code_lv2'].isna()]  

In [ ]:
field = ['Profiles']
id_col_list = ['loan_code_lv2', 'cccd', 'CommonData.CBContractCode', 'process_date']

ts_card = parse_data('ts_card', contract_level_card, field, id_col_list)
ts_install = parse_data('ts_ins', contract_level_install, field, id_col_list)
ts_noninstall = parse_data('ts_nonins', contract_level_noninstall, field, id_col_list)

In [ ]:
ts_card

In [ ]:
df_root.head()